In [1]:
import sys
import os
import xarray
import openpyxl
import contextily as cx 
import yaml
import numpy as np
import pandas
import pathlib
import time
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
run_file="/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/data/whidbey/DOXG/wqm_baseline/daily_min_DOXG.nc"
case = 'whidbey'
run_type = 'wqm_baseline'
model_var = 'DOXG'

In [3]:
with open('../etc/SSM_config_whidbey.yaml', 'r') as file:
    ssm = yaml.safe_load(file)
# get shapefile path    
shp = ssm['paths']['shapefile']
# load shapefile into geopandas dataframe
gdf = gpd.read_file(shp)
DO_thresh = gdf['DO_std']
DO_thresh.shape

(4144,)

In [4]:
impairment=-0.25
# Convert impairment to text string to use in file name
impaired_txt = str(impairment)
impaired_txt = impaired_txt.replace('.','p')
impaired_txt = impaired_txt.replace('-','m')

# Load minimum DO results from scenario
MinDO_full={}
MinDO={}
try: 
    with xarray.open_dataset(run_file) as ds:
        print([*ds])
        MinDO_full[run_type]=ds[f'DOXG_daily_min']
        # Sub-sample nodes (from 16012 nodes to 7494)
        MinDO[run_type]=MinDO_full[run_type][:,:,gdf['node_id']-1]
        print(MinDO[run_type].shape)
except FileNotFoundError:
    print(f'File Not Found: {run_file}')
    
# Load minimum DO results from reference case
reference = ssm['run_information']['reference']
base_dir = '/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/'
reference_file = f'{base_dir}/data/{case}/DOXG/{reference}/daily_min_DOXG.nc'
with xarray.open_dataset(reference_file) as ds:
    MinDO_full[reference]=ds[f'DOXG_daily_min']
    # Sub-sample nodes (from 16012 nodes to 7494)
    MinDO[reference]=MinDO_full[reference][:,:,gdf['node_id']-1]

['DOXG_daily_min']
(361, 10, 4144)


In [5]:
reference

'wqm_reference'

In [6]:
ssm['paths']

{'graphics': '/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/graphics/',
 'model_output': {'whidbey': ['/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/wqm_baseline/ssm_output.nc',
   '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/wqm_reference/ssm_output.nc',
   '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3b/ssm_output.nc',
   '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3c/ssm_output.nc',
   '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3e/ssm_output.nc',
   '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3f/ssm_output.nc',
   '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3g/ssm_output.nc',
   '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3h/ssm_output.nc',
   '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3i/ssm_output.nc',
   '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3l/ssm_output.nc',
   '/mmfs1/gscratch/ssmc/USRS/PSI/Sukyong/kingcounty/3m/ssm_output.nc']},
 'movies': '/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/movi

In [7]:
processed_netcdf_dir = pathlib.Path(ssm['paths']['graphics'])/case/model_var
output_directory = processed_netcdf_dir/run_type/'movies'/'impairment_region'/impaired_txt

In [8]:
output_directory

PosixPath('/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/KingCounty/graphics/whidbey/DOXG/wqm_baseline/movies/impairment_region/m0p25')

In [28]:
# Get number of days and nodes
[ndays,nlevels,nnodes]=MinDO[run_type].shape

# Calculate volume for volume days
volume = np.asarray(gdf.volume)
depth_fraction = np.array(ssm['siglev_diff'])/100
volume2D = np.dot(volume.reshape(nnodes,1),depth_fraction.reshape(1,nlevels))
volume3D = np.repeat(volume2D.transpose()[np.newaxis, :, :], 361, axis=0)

# Initialize dictionaries
DO_diff_lt_0p2={} # Boolean where DO<threshold
DO_diff_lt_0p2_wc = {} # Boolean True where impaired at any level

# Calculate impairment
print(f'Calculating difference for {run_type}')
# Create array of Dissolved Oxygen threshold values 
DO_diff = MinDO[run_type] - MinDO[reference]
# Boolean where DO_diff < -0.2 (or impairment value)
# 361x4144 (nodes x time) or 361x10x4144
DO_diff_lt_0p2[run_type] = DO_diff<=impairment 
# Take max over depth level to flag node as impaired if impaired anywhere in 
# water column
DO_diff_lt_0p2_wc[run_type]=DO_diff_lt_0p2[run_type].max(
    axis=1, initial=0)

Calculating difference for wqm_baseline


In [ ]:
gdf[['region_inf','node_id']].groupby('region_inf').count()

In [10]:
dti = pandas.date_range("2014-01-01", periods=366, freq="D")


In [26]:
f'{dti[0].day:02d}'

'01'

In [27]:
print(f'{dti[0].month_name()} {dti[0].day:02d}, 2014')

January 01, 2014


In [1]:
# Plot impairment for each day

# create date vector
for day in [0,260]:#(ndays):

    
    gdf['Impaired']=DO_diff_lt_0p2_wc[run_type][day,:]
    gdf_impaired = gdf.loc[
        ((gdf['included_i']==1) & 
        (gdf['Impaired']==True) &
        (gdf['region_inf']=='Whidbey'))
    ]
    gdf_good = gdf.loc[
        ((gdf['included_i']==1) & 
        (gdf['Impaired']==False) & 
        (gdf['region_inf']=='Whidbey'))
    ]
    fig, axs = plt.subplots(1,1, figsize = (8,8))
    #~~~ Impaired (red) and Not Impaired (blue) nodes ~~~
    gdf_good.plot(ax=axs,color='blue',legend=True,
                     label='Not Impaired', alpha=0.3)
    if gdf_impaired.empty == False: # plot if there are impaired values
        gdf_impaired.plot(ax=axs,color='red',legend=True,
                         label='Impaired ($\Delta$DO<-0.2)')
    #~~~ Location map ~~~
    cx.add_basemap(axs, crs=gdf.crs, source=cx.providers.Stamen.TerrainBackground, alpha=1) 
    axs.set_title(f'Current Conditions\n Impaired nodes for day {model_day} of 2014')
    #axs.set_xticklabels('')
    #axs.set_yticklabels('')
    #output_file = output_directory/f'{case}_{run_tag}_all_impairment_wc_{model_day}.png'
    #plt.savefig(output_file, bbox_inches='tight', format='png')
    plt.show() #clear figure and memory


NameError: name 'DO_diff_lt_0p2_wc' is not defined


https://stackoverflow.com/questions/56559520/change-background-map-for-contextily
### Tile provider sources ###

ST_TONER = 'http://tile.stamen.com/toner/tileZ/tileX/tileY.png'
ST_TONER_HYBRID = 'http://tile.stamen.com/toner-hybrid/tileZ/tileX/tileY.png'
ST_TONER_LABELS = 'http://tile.stamen.com/toner-labels/tileZ/tileX/tileY.png'
ST_TONER_LINES = 'http://tile.stamen.com/toner-lines/tileZ/tileX/tileY.png'
ST_TONER_BACKGROUND = 'http://tile.stamen.com/toner-background/tileZ/tileX/tileY.png'
ST_TONER_LITE = 'http://tile.stamen.com/toner-lite/tileZ/tileX/tileY.png'

ST_TERRAIN = 'http://tile.stamen.com/terrain/tileZ/tileX/tileY.png'
ST_TERRAIN_LABELS = 'http://tile.stamen.com/terrain-labels/tileZ/tileX/tileY.png'
ST_TERRAIN_LINES = 'http://tile.stamen.com/terrain-lines/tileZ/tileX/tileY.png'
ST_TERRAIN_BACKGROUND = 'http://tile.stamen.com/terrain-background/tileZ/tileX/tileY.png'

ST_WATERCOLOR = 'http://tile.stamen.com/watercolor/tileZ/tileX/tileY.png'

# OpenStreetMap as an alternative
OSM_A = 'http://a.tile.openstreetmap.org/tileZ/tileX/tileY.png'
OSM_B = 'http://b.tile.openstreetmap.org/tileZ/tileX/tileY.png'
OSM_C = 'http://c.tile.openstreetmap.org/tileZ/tileX/tileY.png'

In [ ]:
cx.providers.OpenStreetMap.Mapnik.name.